In [ ]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
from typing import List, Tuple, Dict
import numpy as np

In [ ]:
model = EncoderClassifier.from_hparams(source="speechbrain/lang-id-voxlingua107-ecapa", savedir="ckpt")

In [ ]:
import os

DATA_DIR = 'data'

# Iterate through the folder and save .wav filenames to a list
wav_files = [file for file in os.listdir(DATA_DIR) if file.endswith('.wav')]

In [ ]:
print(wav_files)

In [ ]:
with open('labels.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
len(labels)

In [1]:
# TODO: implement this first
def get_ground_truth_labels(wav_names: List[str]):
  name_to_gt = {}
  for wav_name in wav_names:
    

IndentationError: expected an indented block (3352378936.py, line 5)

In [ ]:
# wav = model.load_audio("/Users/tesnik/Desktop/Workspace/notebooks/spoken_language_identification/data/example_.wav")
# prediction =  model.classify_batch(signal)

# emb =  model.encode_batch(signal)
# print(emb.shape)

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

def plot_mel_spectrogram(wav_file_path):
  y, sr = librosa.load(wav_file_path, sr=None)

  S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)

  log_S = librosa.power_to_db(S, ref=np.max)

  plt.figure(figsize=(10, 4))
  librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel', cmap='coolwarm')
  plt.title('Mel spectrogram')
  plt.colorbar(format='%+02.0f dB')
  plt.tight_layout()
  plt.show()

In [ ]:
plot_mel_spectrogram(f'data/{wav_files[0]}')